In [1]:
import sympy as sp
from sympy import MatrixSymbol, Symbol, symbols
from block_matrix import BlockMatrix, BlockVector, DiagonalBlockMatrix

# Create dimension symbol
n = symbols('n')
inner_dim = n - 1

# Create symbolic matrices with abstract dimension n
m1 = MatrixSymbol('A', inner_dim, inner_dim)  # (n-1)x(n-1) symbolic matrix
m2 = MatrixSymbol('b', inner_dim, 1)          # (n-1)x1 symbolic matrix
m3 = MatrixSymbol('c', 1, inner_dim)          # 1x(n-1) symbolic matrix
m4 = Symbol('d')                              # scalar symbol

# Create block matrix
M = BlockMatrix(m1, m2, m3, m4, n=n)

# Create another symbolic block matrix
p1 = MatrixSymbol('P', inner_dim, inner_dim)
p2 = MatrixSymbol('q', inner_dim, 1)
p3 = MatrixSymbol('r', 1, inner_dim)
p4 = Symbol('s')

N = BlockMatrix(p1, p2, p3, p4, n=n)

# Perform operations
sum_matrix = M + N
product_matrix = M * N
inverse_matrix = M.inverse()

print("Sum of matrices:")
print(sum_matrix)
print("\nProduct of matrices:")
print(product_matrix)
print("\nInverse of first matrix:")
print(inverse_matrix) 

Sum of matrices:
BlockMatrix(
A + P,
b + q,
c + r,
d + s)

Product of matrices:
BlockMatrix(
A*P + b*r,
s*b + A*q,
d*r + c*P,
d*s + Trace(c*q))

Inverse of first matrix:
BlockMatrix(
A**(-1) + 1/(d - Trace(c*A**(-1)*b))*A**(-1)*b*c*A**(-1),
(-1/(d - Trace(c*A**(-1)*b)))*A**(-1)*b,
(-1/(d - Trace(c*A**(-1)*b)))*c*A**(-1),
1/(d - Trace(c*A**(-1)*b)))


In [2]:
v_fix = MatrixSymbol('v', inner_dim, 1)      # (n-1)x1 vector
x = Symbol('x')                              # scalar for last element

# Create block vector
v = BlockVector(v_fix, x, n=n)

# Compute matrix-vector product
result = M * v

print("Block Matrix M:")
print(M)
print("\nBlock Vector v:")
print(v)
print("\nMatrix-vector product M*v:")
print(result)

# Let's also test vector addition and inner product
v2_fix = MatrixSymbol('w', inner_dim, 1)
y = Symbol('y')
v2 = BlockVector(v2_fix, y, n=n)

# Vector addition
sum_vec = v + v2

# Inner product
inner_prod = v.inner_product(v2)

Block Matrix M:
BlockMatrix(
A,
b,
c,
d)

Block Vector v:
BlockVector(
v,
x)

Matrix-vector product M*v:
BlockVector(
x*b + A*v,
d*x + Trace(c*v))


## Setting up variables for the problem

In [3]:
# Create dimension symbol if not already created
n = symbols('n')
inner_dim = n - 1

# Create components for block matrix K
K1 = MatrixSymbol('K1', inner_dim, inner_dim)  # (n-1)x(n-1) matrix
K2 = MatrixSymbol('K2', inner_dim, 1)          # (n-1)x1 matrix 
K3 = MatrixSymbol('K3', 1, inner_dim)          # 1x(n-1) matrix
K4 = Symbol('K4')                              # scalar

# Create block matrix K
K = BlockMatrix(K1, K2, K3, K4, n=n)

# Create diagonal matrices J and J_1
J_fixed = MatrixSymbol('J_fixed', inner_dim, inner_dim)
j_last = Symbol('j_last')
j1_last = Symbol('j1_last')

J = DiagonalBlockMatrix(J_fixed, j_last, n=n)
J_1 = DiagonalBlockMatrix(J_fixed, j1_last, n=n)

# Create vectors y, y_1 and m, m_1 with shared initial parts
y_fixed = MatrixSymbol('y_fixed', inner_dim, 1)
m_fixed = MatrixSymbol('m_fixed', inner_dim, 1)

y_last = Symbol('y_last')
y1_last = Symbol('y1_last')
m_last = Symbol('m_last')
m1_last = Symbol('m1_last')

y = BlockVector(y_fixed, y_last, n=n)
y_1 = BlockVector(y_fixed, y1_last, n=n)
m = BlockVector(m_fixed, m_last, n=n)
m_1 = BlockVector(m_fixed, m1_last, n=n)

In [4]:
def compute_mu_R(K: BlockMatrix, y: BlockVector, m: BlockVector, sigma_sq: Symbol, J_w: DiagonalBlockMatrix) -> BlockVector:
    """
    Compute µᴿ = m + K(K + σ²Jw)⁻¹(y - m)
    
    Parameters:
    - K: The kernel matrix
    - y: The observation vector
    - m: The mean vector
    - sigma_sq: The noise variance σ²
    - J_w: The diagonal matrix Jw
    
    Returns:
    - BlockVector representing µᴿ
    """
    # First compute y - m
    diff = y - m
    
    # Then compute (K + σ²Jw)
    K_plus_sigmaSqJ = K + sigma_sq * J_w
    
    # Compute its inverse
    K_plus_sigmaSqJ_inv = K_plus_sigmaSqJ.inverse()
    
    # Compute K(K + σ²Jw)⁻¹(y - m)
    temp = K_plus_sigmaSqJ_inv * diff
    mu_part = K * temp
    
    # Finally add m to get µᴿ
    return m + mu_part

# Test the computation
print("Computing µᴿ and µᴿ₁ - µᴿ:")
sigma_sq = Symbol('sigma_sq')  # noise variance

# Compute µᴿ
mu_R = compute_mu_R(K, y, m, sigma_sq, J)
print("\nµᴿ = m + K(K + σ²Jw)⁻¹(y - m):")
print(mu_R)

# Compute µᴿ₁
mu_R1 = compute_mu_R(K, y_1, m_1, sigma_sq, J_1)

# Compute their difference
mu_R_diff = mu_R1 - mu_R
print("\nµᴿ₁ - µᴿ:")
print(mu_R_diff)

Computing µᴿ and µᴿ₁ - µᴿ:

µᴿ = m + K(K + σ²Jw)⁻¹(y - m):
BlockVector(
((-m_last + y_last)/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2)) - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*(-m_fixed + y_fixed))/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2)))*K2 + K1*((-(-m_last + y_last)/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2)))*(sigma_sq*J_fixed + K1)**(-1)*K2 + ((sigma_sq*J_fixed + K1)**(-1) + 1/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2))*(sigma_sq*J_fixed + K1)**(-1)*K2*K3*(sigma_sq*J_fixed + K1)**(-1))*(-m_fixed + y_fixed)) + m_fixed,
K4*((-m_last + y_last)/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2)) - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*(-m_fixed + y_fixed))/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2))) + m_last + Trace(K3*((-(-m_last + y_last)/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2)))*(sigma_sq*J_fixed + K1)**(-1)*K2 + ((

In [5]:
def compute_Sigma_R(K: BlockMatrix, sigma_sq: Symbol, J_w: DiagonalBlockMatrix) -> BlockMatrix:
    """
    Compute Σᴿ = K(K + σ²Jw)⁻¹σ²Jw
    
    Parameters:
    - K: The kernel matrix
    - sigma_sq: The noise variance σ²
    - J_w: The diagonal matrix Jw
    
    Returns:
    - BlockMatrix representing Σᴿ
    """
    # First compute σ²Jw
    sigma_sq_J = sigma_sq * J_w
    
    # Then compute (K + σ²Jw)
    K_plus_sigmaSqJ = K + sigma_sq_J
    
    # Compute its inverse
    K_plus_sigmaSqJ_inv = K_plus_sigmaSqJ.inverse()
    
    # Compute K(K + σ²Jw)⁻¹σ²Jw
    temp = K_plus_sigmaSqJ_inv * sigma_sq_J
    return K * temp

# Test the computation
print("Computing Σᴿ and Σᴿ₁:")
sigma_sq = Symbol('sigma_sq')  # noise variance

# Compute Σᴿ
Sigma_R = compute_Sigma_R(K, sigma_sq, J)
print("\nΣᴿ = K(K + σ²Jw)⁻¹σ²Jw:")
print(Sigma_R)

# Compute Σᴿ₁
Sigma_R1 = compute_Sigma_R(K, sigma_sq, J_1)
print("\nΣᴿ₁ = K(K + σ²Jw₁)⁻¹σ²Jw₁:")
print(Sigma_R1)

Computing Σᴿ and Σᴿ₁:

Σᴿ = K(K + σ²Jw)⁻¹σ²Jw:
BlockMatrix(
sigma_sq*K1*((sigma_sq*J_fixed + K1)**(-1) + 1/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2))*(sigma_sq*J_fixed + K1)**(-1)*K2*K3*(sigma_sq*J_fixed + K1)**(-1))*J_fixed + (-sigma_sq/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2)))*K2*K3*(sigma_sq*J_fixed + K1)**(-1)*J_fixed,
(-j_last*sigma_sq/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2)))*K1*(sigma_sq*J_fixed + K1)**(-1)*K2 + (j_last*sigma_sq/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2)))*K2,
sigma_sq*K3*((sigma_sq*J_fixed + K1)**(-1) + 1/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2))*(sigma_sq*J_fixed + K1)**(-1)*K2*K3*(sigma_sq*J_fixed + K1)**(-1))*J_fixed + (-K4*sigma_sq/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2)))*K3*(sigma_sq*J_fixed + K1)**(-1)*J_fixed,
K4*j_last*sigma_sq/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2)) - j_

In [7]:
def compute_Sigma_R_inverse(K: BlockMatrix, sigma_sq: Symbol, J_w: DiagonalBlockMatrix) -> BlockMatrix:
    """
    Compute Σᴿ⁻¹ = (1/σ²)J⁻¹(K + σ²J)K⁻¹
    
    Parameters:
    - K: The kernel matrix
    - sigma_sq: The noise variance σ²
    - J_w: The diagonal matrix Jw
    
    Returns:
    - BlockMatrix representing Σᴿ⁻¹
    """
    # First compute K⁻¹
    K_inv = K.inverse()
    
    # Compute J⁻¹
    J_inv = J_w.inverse()
    
    # Compute K + σ²J
    K_plus_sigmaSqJ = K + sigma_sq * J_w
    
    # Create 1/σ² as a proper Symbol
    inv_sigma_sq = 1/sigma_sq
    
    # Compute (1/σ²)J⁻¹(K + σ²J)K⁻¹
    temp = K_plus_sigmaSqJ * K_inv
    temp = J_inv * temp
    return inv_sigma_sq * temp

# Test the computation
print("Computing Σᴿ⁻¹ and Σᴿ₁⁻¹:")
sigma_sq = Symbol('sigma_sq')  # noise variance

# Compute Σᴿ⁻¹
Sigma_R_inv = compute_Sigma_R_inverse(K, sigma_sq, J)
print("\nΣᴿ⁻¹ = (1/σ²)J⁻¹(K + σ²J)K⁻¹:")
print(Sigma_R_inv)

# Compute Σᴿ₁⁻¹
Sigma_R1_inv = compute_Sigma_R_inverse(K, sigma_sq, J_1)
print("\nΣᴿ₁⁻¹ = (1/σ²)J₁⁻¹(K + σ²J₁)K⁻¹:")
print(Sigma_R1_inv)

Computing Σᴿ⁻¹ and Σᴿ₁⁻¹:

Σᴿ⁻¹ = (1/σ²)J⁻¹(K + σ²J)K⁻¹:
BlockMatrix(
1/sigma_sq*J_fixed**(-1)*((-1/(K4 - Trace(K3*K1**(-1)*K2)))*K2*K3*K1**(-1) + (sigma_sq*J_fixed + K1)*(K1**(-1) + 1/(K4 - Trace(K3*K1**(-1)*K2))*K1**(-1)*K2*K3*K1**(-1))),
1/sigma_sq*J_fixed**(-1)*((-1/(K4 - Trace(K3*K1**(-1)*K2)))*(sigma_sq*J_fixed + K1)*K1**(-1)*K2 + 1/(K4 - Trace(K3*K1**(-1)*K2))*K2),
(1/(j_last*sigma_sq))*((-(K4 + j_last*sigma_sq)/(K4 - Trace(K3*K1**(-1)*K2)))*K3*K1**(-1) + K3*(K1**(-1) + 1/(K4 - Trace(K3*K1**(-1)*K2))*K1**(-1)*K2*K3*K1**(-1))),
((K4 + j_last*sigma_sq)/(K4 - Trace(K3*K1**(-1)*K2)) - Trace(K3*K1**(-1)*K2)/(K4 - Trace(K3*K1**(-1)*K2)))/(j_last*sigma_sq))

Σᴿ₁⁻¹ = (1/σ²)J₁⁻¹(K + σ²J₁)K⁻¹:
BlockMatrix(
1/sigma_sq*J_fixed**(-1)*((-1/(K4 - Trace(K3*K1**(-1)*K2)))*K2*K3*K1**(-1) + (sigma_sq*J_fixed + K1)*(K1**(-1) + 1/(K4 - Trace(K3*K1**(-1)*K2))*K1**(-1)*K2*K3*K1**(-1))),
1/sigma_sq*J_fixed**(-1)*((-1/(K4 - Trace(K3*K1**(-1)*K2)))*(sigma_sq*J_fixed + K1)*K1**(-1)*K2 + 1/(K4 - Trace(K3*K

## Boss term

In [9]:
temp = Sigma_R_inv * mu_R_diff
result_1 = mu_R_diff.inner_product(temp)
result_1

(((K4 + j_last*sigma_sq)/(K4 - Trace(K3*K1**(-1)*K2)) - Trace(K3*K1**(-1)*K2)/(K4 - Trace(K3*K1**(-1)*K2)))*(K4*((-m1_last + y1_last)/(K4 + j1_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2)) - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*(-m_fixed + y_fixed))/(K4 + j1_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2))) - K4*((-m_last + y_last)/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2)) - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*(-m_fixed + y_fixed))/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2))) + m1_last - m_last + Trace(K3*((-(-m1_last + y1_last)/(K4 + j1_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2)))*(sigma_sq*J_fixed + K1)**(-1)*K2 + ((sigma_sq*J_fixed + K1)**(-1) + 1/(K4 + j1_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + K1)**(-1)*K2))*(sigma_sq*J_fixed + K1)**(-1)*K2*K3*(sigma_sq*J_fixed + K1)**(-1))*(-m_fixed + y_fixed))) - Trace(K3*((-(-m_last + y_last)/(K4 + j_last*sigma_sq - Trace(K3*(sigma_sq*J_fixed + 

## Simplifying the term

In [13]:
from sympy import Matrix, Symbol, ones

# Set n=3, so inner_dim = 2
n = 3
inner_dim = 2

# Create K1 as a 2x2 matrix with values that make it invertible
K1_matrix = Matrix([[2, 1], [1, 2]])  # This is a symmetric positive definite matrix

# Create substitution dictionary
subs_dict = {
    # Matrix blocks
    K1: K1_matrix,      # 2x2 matrix [[2,1],[1,2]]
    K2: Matrix([[1], [1]]),     # 2x1 matrix of ones
    K3: Matrix([[1, 1]]),     # 1x2 matrix of ones
    K4: 2,              # scalar
    
    # Diagonal matrices - make it diagonal to ensure invertibility
    J_fixed: Matrix([[2, 0], [0, 2]]), # 2x2 diagonal matrix
    j_last: 2,          # scalar
    # j1_last is kept symbolic
    
    # Vectors
    y_fixed: Matrix([[1], [1]]), # 2x1 matrix of ones
    m_fixed: Matrix([[1], [1]]), # 2x1 matrix of ones
    
    y_last: 1,          # scalar
    m_last: 1,          # scalar
    # m1_last is kept symbolic
    # y1_last is kept symbolic
    
    # Additional symbols
    sigma_sq: 2,        # noise variance
}

# Apply substitutions to the expression
simplified_result = result_1.subs(subs_dict)
print("Simplified expression:")
print(simplified_result)

Simplified expression:
((-Trace(Matrix([[1, 1]])*Matrix([
[2, 1],
[1, 2]])**(-1)*Matrix([
[1],
[1]]))/(2 - Trace(Matrix([[1, 1]])*Matrix([
[2, 1],
[1, 2]])**(-1)*Matrix([
[1],
[1]]))) + 6/(2 - Trace(Matrix([[1, 1]])*Matrix([
[2, 1],
[1, 2]])**(-1)*Matrix([
[1],
[1]]))))*(m1_last + 2*(-m1_last + y1_last)/(2*j1_last - Trace(Matrix([[1, 1]])*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1)*Matrix([
[1],
[1]])) + 2) - Trace(Matrix([[1, 1]])*(((2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1) + 1/(6 - Trace(Matrix([[1, 1]])*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1)*Matrix([
[1],
[1]])))*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1)*Matrix([
[1],
[1]])*Matrix([[1, 1]])*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1))*(-Matrix([
[1],
[1]]) + Matrix([
[1],
[1]])) + 0*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1)*Matrix([
[1],
[1]]))) + Trace(Matrix([[1, 1]])*((-(-m1_last + y1_last)/(2*j1_last

In [14]:
# Evaluate the trace terms explicitly
K1_inv = Matrix([[2, 1], [1, 2]]).inv()
ones_vec = Matrix([[1], [1]])
ones_row = Matrix([[1, 1]])

# Basic trace: Trace(Matrix([[1, 1]])*Matrix([[2, 1], [1, 2]])**(-1)*Matrix([[1], [1]]))
basic_trace = (ones_row * K1_inv * ones_vec)[0]  # The [0] gets the scalar value
print("Basic trace value:", basic_trace)

# For terms with (2*Matrix([[2, 0], [0, 2]]) + Matrix([[2, 1], [1, 2]]))
J_fixed = Matrix([[2, 0], [0, 2]])
K_plus_sigmaJ = 2*J_fixed + Matrix([[2, 1], [1, 2]])
K_plus_sigmaJ_inv = K_plus_sigmaJ.inv()
other_trace = (ones_row * K_plus_sigmaJ_inv * ones_vec)[0]

print("Other trace value:", other_trace)

# Now substitute these values in the expression
j1_last = Symbol('j1_last')
m1_last = Symbol('m1_last')
y1_last = Symbol('y1_last')

# Create a new substitution dictionary with the evaluated traces
trace_subs = {
    # Replace any Trace(...) expressions with their numerical values
    # We'll see the actual values after running this cell
    basic_trace: basic_trace.evalf(),  # Evaluate to numerical value
    other_trace: other_trace.evalf()   # Evaluate to numerical value
}

# Apply the substitutions to get a simpler expression
simpler_result = simplified_result.subs(trace_subs)
print("\nSimplified expression with evaluated traces:")
print(simpler_result)

Basic trace value: 2/3
Other trace value: 2/7

Simplified expression with evaluated traces:
((-Trace(Matrix([[1, 1]])*Matrix([
[2, 1],
[1, 2]])**(-1)*Matrix([
[1],
[1]]))/(2 - Trace(Matrix([[1, 1]])*Matrix([
[2, 1],
[1, 2]])**(-1)*Matrix([
[1],
[1]]))) + 6/(2 - Trace(Matrix([[1, 1]])*Matrix([
[2, 1],
[1, 2]])**(-1)*Matrix([
[1],
[1]]))))*(m1_last + 2*(-m1_last + y1_last)/(2*j1_last - Trace(Matrix([[1, 1]])*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1)*Matrix([
[1],
[1]])) + 2) - Trace(Matrix([[1, 1]])*(((2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1) + 1/(6 - Trace(Matrix([[1, 1]])*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1)*Matrix([
[1],
[1]])))*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1)*Matrix([
[1],
[1]])*Matrix([[1, 1]])*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1))*(-Matrix([
[1],
[1]]) + Matrix([
[1],
[1]])) + 0*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1)*Matrix([
[1]

In [15]:
simpler_result

((-Trace(Matrix([[1, 1]])*Matrix([
[2, 1],
[1, 2]])**(-1)*Matrix([
[1],
[1]]))/(2 - Trace(Matrix([[1, 1]])*Matrix([
[2, 1],
[1, 2]])**(-1)*Matrix([
[1],
[1]]))) + 6/(2 - Trace(Matrix([[1, 1]])*Matrix([
[2, 1],
[1, 2]])**(-1)*Matrix([
[1],
[1]]))))*(m1_last + 2*(-m1_last + y1_last)/(2*j1_last - Trace(Matrix([[1, 1]])*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1)*Matrix([
[1],
[1]])) + 2) - Trace(Matrix([[1, 1]])*(((2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1) + 1/(6 - Trace(Matrix([[1, 1]])*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1)*Matrix([
[1],
[1]])))*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1)*Matrix([
[1],
[1]])*Matrix([[1, 1]])*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1))*(-Matrix([
[1],
[1]]) + Matrix([
[1],
[1]])) + 0*(2*Matrix([
[2, 0],
[0, 2]]) + Matrix([
[2, 1],
[1, 2]]))**(-1)*Matrix([
[1],
[1]]))) + Trace(Matrix([[1, 1]])*((-(-m1_last + y1_last)/(2*j1_last - Trace(Matrix([[1, 1]

In [27]:
print(sp.simplify(sp.expand(sp.simplify(simplified_result))))

(49*j1_last**2*m1_last**2 - 98*j1_last**2*m1_last + 49*j1_last**2 + 14*j1_last*m1_last**2 + 70*j1_last*m1_last*y1_last - 98*j1_last*m1_last - 70*j1_last*y1_last + 84*j1_last + 6*m1_last**2 - 12*m1_last + 30*y1_last**2 - 60*y1_last + 36)/(49*j1_last**2 + 84*j1_last + 36)


In [28]:
from sympy import expand, simplify, Symbol, collect

# Define symbols
j1_last = Symbol('j1_last')
m1_last = Symbol('m1_last')
y1_last = Symbol('y1_last')

# Original expression numerator
original_num = (49*j1_last**2*m1_last**2 - 98*j1_last**2*m1_last + 49*j1_last**2 
               + 14*j1_last*m1_last**2 + 70*j1_last*m1_last*y1_last - 98*j1_last*m1_last 
               - 70*j1_last*y1_last + 84*j1_last 
               + 6*m1_last**2 - 12*m1_last + 30*y1_last**2 - 60*y1_last + 36)

# Original expression denominator
original_den = 49*j1_last**2 + 84*j1_last + 36

# My proposed factored form
d = 7*j1_last + 6
a = 7*j1_last*(m1_last - 1)**2 + 10*y1_last*(m1_last - 1) - 6*(m1_last - 1)
b = 30*(y1_last - 1)**2

# Expand my factored form
factored_num = a*d + b

print("Original numerator:")
print(original_num)
print("\nExpanded factored form:")
print(expand(factored_num))
print("\nAre they equal?")
print(simplify(original_num - expand(factored_num)) == 0)

# Show the expression in a/d + b/d^2 form
print("\nExpression in a/d + b/d^2 form:")
print(f"a = {a}")
print(f"b = {b}")
print(f"d = {d}")

Original numerator:
49*j1_last**2*m1_last**2 - 98*j1_last**2*m1_last + 49*j1_last**2 + 14*j1_last*m1_last**2 + 70*j1_last*m1_last*y1_last - 98*j1_last*m1_last - 70*j1_last*y1_last + 84*j1_last + 6*m1_last**2 - 12*m1_last + 30*y1_last**2 - 60*y1_last + 36

Expanded factored form:
49*j1_last**2*m1_last**2 - 98*j1_last**2*m1_last + 49*j1_last**2 + 42*j1_last*m1_last**2 + 70*j1_last*m1_last*y1_last - 126*j1_last*m1_last - 70*j1_last*y1_last + 84*j1_last + 60*m1_last*y1_last - 36*m1_last + 30*y1_last**2 - 120*y1_last + 66

Are they equal?
False

Expression in a/d + b/d^2 form:
a = 7*j1_last*(m1_last - 1)**2 - 6*m1_last + 10*y1_last*(m1_last - 1) + 6
b = 30*(y1_last - 1)**2
d = 7*j1_last + 6


In [30]:
from sympy import expand, factor, Symbol, collect, apart

# Define symbols
j1_last = Symbol('j1_last')
m1_last = Symbol('m1_last')
y1_last = Symbol('y1_last')

# Original expression
numerator = (49*j1_last**2*m1_last**2 - 98*j1_last**2*m1_last + 49*j1_last**2 
            + 14*j1_last*m1_last**2 + 70*j1_last*m1_last*y1_last - 98*j1_last*m1_last 
            - 70*j1_last*y1_last + 84*j1_last 
            + 6*m1_last**2 - 12*m1_last + 30*y1_last**2 - 60*y1_last + 36)
denominator = 49*j1_last**2 + 84*j1_last + 36

# Full expression
expr = numerator/denominator

# Let's try different approaches to factor:

# 1. Try partial fraction decomposition
print("Partial fraction decomposition:")
simplified_expr_2 = apart(expr, j1_last)
simplified_expr_2

Partial fraction decomposition:


m1_last**2 - 2*m1_last + 1 - 10*(m1_last - 1)*(m1_last - y1_last)/(7*j1_last + 6) + 30*(m1_last - y1_last)**2/(7*j1_last + 6)**2